In [12]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [11]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

In [49]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [89]:
from src.run import H_CONFIG, IMAGE_GPT_CONFIGS, set_hparams, create_model
from src.utils import AttrDict, load_pickle
from src.model import model

In [106]:
_here = Path('.').resolve()
dataset = load_pickle(Path('~/scratch/attila/results/minGPT_brain_data.pkl').expanduser())
X = np.expand_dims(dataset, axis=-1)[:16, ...]  # grayscale images => 1 channel

X_train = X.flatten().reshape(X.shape[0], X.shape[1] * X.shape[2], X.shape[-1])[..., 0]

print('X ~', X.shape)
print('X feeded ~', X_train.shape)

X ~ (16, 32, 32, 1)
X feeded ~ (16, 1024)


In [95]:
args = AttrDict({ **H_CONFIG, **IMAGE_GPT_CONFIGS['s'] })
hparams = set_hparams(args)

set_seed(args.seed)
n_batch = args.n_sub_batch * args.n_gpu
n_class = 0  # NOT fine-tuning (for the moment)

In [109]:
tf.reset_default_graph()
x = tf.placeholder(tf.int32, [n_batch, args.n_px * args.n_px])

results = model(hparams, x, reuse=False)
# trainable_params = tf.trainable_variables()

loss = tf.reduce_mean(results['gen_loss'])  # if needed, when there are many GPUs

optimizer = tf.train.AdamOptimizer(learning_rate=2e-5)
train_op = optimizer.minimize(loss)

In [113]:
n_batch = args.n_sub_batch * args.n_gpu
# saver = tf.train.Saver(var_list=trainable_params)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

metrics = []
for x_batch in iter_data(X_train, n_batch=n_batch, verbose=True):
    print(x_batch.shape)
  
    metrics.append(sess.run([ train_op, gen_loss[0] ], feed_dict={x: x_batch}))
        
# with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)) as sess:
#     sess.run(tf.global_variables_initializer())    

(8, 1024)


ValueError: Fetch argument <tf.Tensor 'truediv_19:0' shape=() dtype=float32> cannot be interpreted as a Tensor. (Tensor Tensor("truediv_19:0", shape=(), dtype=float32, device=/device:GPU:0) is not an element of this graph.)

TypeError: Dimension value must be integer or None or have an __index__ method, got value 'inf' with type '<class 'float'>'

In [ ]:
sess = tf.Session()
sess.run(init)  # Run the initializer

for i in range(1, hparams.train_epochs + 1):
    # Get the next batch of MNIST data (only images are needed, not labels)
    batch_x, _ = mnist.train.next_batch(batch_size)

    # Train
    feed_dict = {input_image: batch_x}
    _, l = sess.run([train_op, loss_op], feed_dict=feed_dict)
    if i % 1000 == 0 or i == 1:
        print('Step %i, Loss: %f' % (i, l))